# Set Library Path

In [ ]:
.libPaths("/share/korflab/home/viki/anaconda3/jupyter_nb/lib/R/library")

# Load Libraries

In [ ]:
library(dplyr)
library(rtracklayer)
library(GenomicRanges)
library(tibble)
library(readr)
library(tidyr)

# Process RNA-seq Data

## Load Data

In [ ]:
# Read in normalized gene counts from DEG analysis
counts <- read.delim("rnaseq/05_gene_counts/normalized_counts.txt")

# View
head(counts)

In [ ]:
# Prepare to assign gene names 
names <- rownames(counts) %>% as.data.frame()
colnames(names) <- "gene_names"
rownames(counts) <- NULL

# View
head(counts)

In [ ]:
# Load data
rnaseq <- read.csv("rnaseq/05_DEGs/control_vs_faexcess_genes.csv", header = TRUE) %>%
dplyr::select("gene_names","external_gene_name")

new_names <- names %>% dplyr::left_join(rnaseq, by = "gene_names")

# View
head(new_names)

In [ ]:
# Combine data to get external_gene_name
counts <- cbind(new_names, counts)

# Remove rows where external_gene_name is NA
counts <- counts[!is.na(counts$external_gene_name), ]

# View
head(counts)

In [ ]:
# Assign gene names as row names
rownames(counts) <- counts$external_gene_name

# Remove the gene_names and external_gene_name columns
counts <- counts[, !(names(counts) %in% c("gene_names", "external_gene_name"))]

# View the updated counts data frame
head(counts)

## Convert Gene Lengths to Kilobases for TPM Calculation

In [ ]:
# Read the GTF annotation file
gtf_file <- "/share/lasallelab/genomes/mm10/mm10.refGene.gtf"
gtf_data <- import(gtf_file)

# View
head(gtf_data)

In [ ]:
# Remove rows from counts if genes are not found in the annotation file

# Assign unique genes for easy searching
unique_genes <- unique(gtf_data$gene_name)

# Filter counts_data to keep only rows where the row names are in unique_genes
counts <- counts[rownames(counts) %in% unique_genes, ]

# View
head(counts)
print(length(rownames(counts)))

In [ ]:
# Calculate gene lengths

# Filter for exon features
exon_data <- gtf_data[gtf_data$type == "exon"]

# Calculate the length of each exon
exon_lengths <- width(exon_data)

# Create a data frame to store gene lengths
gene_lengths <- data.frame(gene_id = exon_data$gene_id, length = exon_lengths)

# Sum the lengths for each gene
gene_length_summary <- aggregate(length ~ gene_id, data = gene_lengths, FUN = sum)

# Print the gene lengths
head(gene_length_summary)
print(length(gene_length_summary$gene_id))

In [ ]:
# Convert gene length to kilobases
gene_length_summary <- gene_length_summary %>%
  mutate(length_kb = length / 1000)

head(gene_length_summary)

## Calculate Reads Per Kilobase

In [ ]:
# Convert counts to a data frame 
counts <- as.data.frame(counts)

# Add gene_id as a column to counts
counts$gene_id <- rownames(counts)

# Merge counts with gene_length_summary
merged_data <- merge(counts, gene_length_summary, by = "gene_id", all.x = TRUE)

# Calculate RPK for each gene (RPK = counts / length in kilobases)
count_columns <- names(merged_data)[-which(names(merged_data) %in% c("gene_id", "length_kb"))]

# Divide the count columns by length_kb
merged_data[count_columns] <- merged_data[count_columns] / merged_data$length_kb

# Set the gene_id back as row names
rownames(merged_data) <- merged_data$gene_id

# Clean data
merged_data$gene_id <- NULL
merged_data$length_kb <- NULL
merged_data$length <- NULL

# View the RPK data
head(merged_data)

In [ ]:
# Verify proper RPK calculations
specific_row <- counts["0610005C13Rik", , drop = FALSE]

# Convert the row to a numeric vector
specific_row_vector <- as.numeric(specific_row)

# Divide every value in the row by the gene length (in kb) of 0610005C13Rik
expected_counts <- specific_row_vector / 2.037

# Print the output
print(expected_counts)

## Calculate Total Reads Per Kilobase

In [ ]:
# Calculate the total RPK for each sample 
total_rpk <- colSums(merged_data)

## Calculate TPM per Sample

In [ ]:
# Calculate TPM for each gene (TPM = (RPK / Total RPK) * 1,000,000)
tpm <- sweep(merged_data, 2, total_rpk, FUN = "/") * 1e6

# Convert the result to a data frame (optional)
tpm_df <- as.data.frame(tpm)

# View
head(tpm_df)

In [ ]:
# Save TPM data to a CSV
write.csv(tpm_df, file = "RNAseq_TPM_Values.csv", row.names = TRUE)

In [ ]:
colnames(tpm_df)

# Process WGBS Data

## Annotate Genes

In [ ]:
# Load data
files <- list.files(path = "wgbs/08_cytosine_reports", 
                    pattern = "\\.deduplicated\\.bismark\\.cov\\.gz\\.CpG_report\\.merged_CpG_evidence\\.cov\\.gz$", 
                    full.names = TRUE)

# View the list of files
print(files)

In [ ]:
# Read the GTF annotation file
gtf_file <- "/share/lasallelab/genomes/mm10/mm10.refGene.gtf"
gtf_data <- import(gtf_file)

# View
head(gtf_data)

In [ ]:
gtf_transcripts <- gtf_data[gtf_data$type == "transcript"]

# Create GRanges object for gene annotations
gr_genes <- GRanges(seqnames = seqnames(gtf_transcripts),
                    ranges = IRanges(start = start(gtf_transcripts), end = end(gtf_transcripts)),
                    gene_name = mcols(gtf_transcripts)$gene_name)

# View 
head(gr_genes)

In [ ]:
# Loop through each file in the files list
for (file in files) {
  # Read the gzipped file
  regions <- read.table(gzfile(file), header = FALSE, stringsAsFactors = FALSE)
  
  # Create GRanges object for regions
  gr_regions <- GRanges(seqnames = regions$V1,  
                        ranges = IRanges(start = regions$V2, end = regions$V3))  
  
  # Find overlaps between regions and gene annotations
  overlaps <- findOverlaps(gr_regions, gr_genes)
  
  # Create a new column for gene names in the regions data frame
  regions$gene_name <- NA 
  regions$gene_name[queryHits(overlaps)] <- gr_genes$gene_name[subjectHits(overlaps)]

  # Do not save columns where gene names are NA
  regions <- regions %>% filter(!is.na(gene_name))
  
  # Create a sample basename for saving the results
  sample_basename <- sub("\\..*$", "", basename(file))
  
  # Save the annotated regions to a CSV file
  output_directory <- "wgbs/08_cytosine_reports/"
  write.csv(regions, file = paste0(output_directory, sample_basename, "_annotated_regions.csv"), row.names = FALSE)

  # Print progress
  cat(sprintf("Regions have been assigned for %s...\n", sample_basename))
}

## Calculate Percent Methylation Per Gene

In [ ]:
# Load data
files <- list.files(path = "wgbs/08_cytosine_reports", 
                    pattern = "\\.csv$", 
                    full.names = TRUE)

# View the list of files
print(files)

In [ ]:
# Loop through each file
for (file in files) {
  # Read the CSV file
  data <- read.csv(file, stringsAsFactors = FALSE)
  
  # Aggregate counts by gene_name
  aggregated_data <- data %>%
    group_by(gene_name) %>%
    summarise(
      methylated_cytosines = sum(V5, na.rm = TRUE),
      unmethylated_cytosines = sum(V6, na.rm = TRUE),
      .groups = 'drop'  # This ensures that the grouping is dropped after summarising
    ) %>%
    mutate(
      percent_methylated = (methylated_cytosines / (methylated_cytosines + unmethylated_cytosines)) * 100
    )
    
  # Specify the output directory
  output_directory <- "wgbs/08_cytosine_reports/"
  
  # Extract the first part of the basename before the first underscore
  base_name <- tools::file_path_sans_ext(basename(file))
  first_part <- strsplit(base_name, "_")[[1]][1]
  
  # Save the aggregated data to a CSV file in the specified directory
  write.csv(aggregated_data, file = paste0(output_directory, first_part, "_percent_methylated.csv"), row.names = FALSE)
}

## Create Percent Methylation Table Comparable to TPM Table

In [ ]:
# Load data
files <- list.files(path = "wgbs/08_cytosine_reports", 
                    pattern = "FA\\d+_percent_methylated\\.csv", 
                    full.names = TRUE)

# View the list of files
print(files)

In [ ]:
# Initialize an empty data frame
df <- data.frame()

# Loop through each CSV file
for (file in files) {
  # Read the CSV file
  data <- read.csv(file)
     
  # Delete unnecessary files
  data$methylated_cytosines <- NULL
  data$unmethylated_cytosines <- NULL
    
  # Extract sample names from the file names
  sample_name <- gsub(".*?(FA\\d+)_percent_methylated\\.csv", "\\1", file)

  # Rename percent_methylated to sample name
  colnames(data)[colnames(data) == "percent_methylated"] <- sample_name
    
  # Merge the current data frame with the main data frame
  if (nrow(df) == 0) {
    df <- data  
  } else {
    df <- merge(df, data, by = "gene_name", all = TRUE) 
  }
}

# View
head(df)

In [ ]:
# Assign gene names as row names to match formatting of RNA-seq TPM counts
rownames(df) <- df$gene_name
df$gene_name <- NULL

# View
head(df)

In [ ]:
# Save percent methylation data to a CSV
write.csv(df, file = "WGBS_Percent_Methylation_Values.csv", row.names = TRUE)

# Integrate Data Frames

In [1]:
# Load RNA-seq data
rnaseq <- read.csv("RNAseq_TPM_Values.csv")

# Assign row names
rownames(rnaseq) <- rnaseq$X
rnaseq$X <- NULL

# View
head(rnaseq)

,G1_1_1_4,G1_1_1_5,G1_1_1_6,G1_1_1_7,G1_1_2_4,G1_1_2_5,G1_1_2_6,G2_2_6_5,G2_2_6_6,G2_2_6_7,G2_2_6_8,G2_2_7_10,G2_2_7_11,G2_2_7_7,G2_2_7_8,G2_2_7_9
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0610005C13Rik,41.775010,26.80276852,11.411480,-8.752577,28.067142,17.843883,29.846533,27.205443,32.433327,22.012507,11.805063,34.419602,39.165244,33.380731,29.440247,29.658463
0610009E02Rik,64.230624,56.82052616,72.032676,64.734080,56.006185,58.560116,63.471752,62.794459,61.696110,57.675975,66.559522,66.853333,67.455080,70.571016,62.772558,69.859374
0610009L18Rik,279.959717,236.48884339,229.363575,254.246831,250.882158,241.185135,189.873772,234.380141,214.005398,203.886753,219.292072,212.777777,217.114917,244.029054,270.012105,228.017498
0610010K14Rik,-1.595618,0.07403448,7.975566,4.291156,9.722921,3.935585,1.842714,4.854214,-8.956592,3.323151,8.512191,3.083256,4.270727,6.995823,8.438767,8.415358
0610030E20Rik,46.555287,37.49050991,41.171451,41.249467,41.114246,36.703945,42.770950,43.645316,39.017554,41.895253,44.675313,38.360030,41.524456,44.423049,38.307154,39.312327
0610039K10Rik,162.010043,159.39222526,171.496195,173.124424,165.149861,186.658612,203.857924,191.094286,155.064311,180.176168,174.776664,196.674571,183.785579,177.454793,182.942072,135.496452


In [2]:
# Load WGBS data
wgbs <- read.csv("WGBS_Percent_Methylation_Values.csv")

# Assign row names
rownames(wgbs) <- wgbs$X
wgbs$X <- NULL

# View
head(wgbs)

,FA114,FA115,FA116,FA117,FA124,FA125,FA126,FA265,FA266,FA2710,FA2711,FA277
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0610005C13Rik,34.117647,32.512315,26.337449,17.1052632,27.325581,31.052632,54.000000,26.744186,26.38436,21.014493,30.555556,34.666667
0610009B22Rik,64.814815,57.915058,68.558952,74.2971888,54.545455,66.336634,70.813397,75.362319,71.00977,60.795455,54.861111,61.029412
0610009E02Rik,57.046980,56.688963,61.684211,59.1269841,60.702875,57.035647,61.815068,61.607143,58.15739,55.882353,54.721550,55.555556
0610009L18Rik,2.857143,3.664921,2.279202,0.9111617,2.165605,4.247104,1.355932,1.596517,4.58613,2.716049,2.678571,2.047782
0610010F05Rik,80.000000,80.498084,80.208708,85.0646950,74.821705,78.614351,80.832037,79.350034,74.83296,86.412764,79.834111,77.411477
0610010K14Rik,32.947977,16.049383,28.767123,22.4683544,18.264840,40.086207,24.215247,16.981132,28.30189,50.000000,38.738739,28.521127


# Calculate Correlations

# Visualize Correlations